# Parser for RateBeer

In this notebook, we are parsing the HTML pages from the RateBeer website.

In [3]:
import os
os.chdir('..')

In [4]:
from classes.parser import *

import pandas as pd
import numpy as np
import datetime
import time
import html
import gzip
import re
import os

data_folder = '../data/'

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [5]:
parser = Parser(data_folder)

# Parse the breweries from the places

In [ ]:
parser.parse_breweries_from_places()

# Parse the beers from the breweries

In [ ]:
parser.parse_brewery_files()

# Parse the beer files for information

In [ ]:
parser.parse_beer_files_for_information()

# Parse the beer files for the reviews

In [ ]:
parser.parse_beer_files_for_reviews()

# Get the users from the ratings

In [ ]:
parser.get_users_from_ratings()

# Parse the user files for information

In [ ]:
parser.parse_all_users()

In [1]:
str_ =                     str_ = '<div style="display:inline; padding: 0px 0px; font-size: 24px; font-weight: bold; color: ' \
                           '#036;" title="(.+?) out of 5.0<br /><small>Aroma (\d+)/10<br />Appearance (\d+)/5<br />' \
                           'Taste (\d+)/10<br />Palate (\d+)/5<br />Overall (\d+)/20<br /></small>">(.+?)</div></div>' \
                           '<small style="color: #666666; font-size: 12px; font-weight: bold;"><A HREF="/user/(\d+)/">' \
                           '(.+?)\xa0\((\d+)\)</A></I> -(.+?)- (.+?)</small><BR><div style="padding: 20px 10px 20px ' \
                           '0px; border-bottom: 1px solid #e0e0e0; line-height: 1.5;">(.+?)</div><br>'

In [7]:
# Open the file
html_txt = open('../150375/7.html', 'rb').read().decode('ISO-8859-1')

# Unescape the HTML characters
html_txt = html.unescape(html_txt)

# Remove the \n, \r and \t characters
html_txt = html_txt.replace('\r', '').replace('\n', '').replace('\t', '')

In [20]:
grp = re.finditer(str_, html_txt)

In [21]:
for g in grp:
    print(g.group(12))
    if '<a' in g.group(12):
        asd

JUL 28, 2012
JUN 16, 2012 <a href=/RatingsQA.htm>does not count <IMG height=10 width=10 title="click to see why this rating of N&R - Shot In The Dark does not count" SRC="/images/icons/questionmark.gif" align=middle border=0></a> 


NameError: name 'asd' is not defined

In [31]:
splitted = g.group(12).split(' - ')

In [32]:
idx = 1

In [33]:
while not('20' in splitted[-idx] or '19' in splitted[-idx]):
    idx += 1

In [35]:
str_date = g.group(12).split(' - ')[-idx]

In [39]:
int(str_date.split(', ')[1].split('<')[0])

2012

In [25]:
g.group(12).split(',')[].split('<')[0]

' 2012 '

In [27]:
g.group(12).split(' - ')[-1]

'Shot In The Dark does not count" SRC="/images/icons/questionmark.gif" align=middle border=0></a> '